# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. <br>Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

#### Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

## 1. Изучаю данные

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
df.columns.to_list()

['calls', 'minutes', 'messages', 'mb_used', 'is_ultra']

Данные представлены в 5 колонках, всего 3214 строк, пропущенных значений нет. В 4 столбах содержатся числовые данные типа float64, в одном - категориальная переменная, данные типа int64.

In [6]:
df.is_ultra.value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

## 2. Разбиваю данные на выборки

In [7]:
# < разделяю данные на обучающую и валидационную выборки >
df_train, df_valid = train_test_split(df, test_size=0.20, stratify = df['is_ultra'], random_state=12345) 
# < создаю переменные для признаков и целевого признака >
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra'] 

In [8]:
features_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2571 entries, 111 to 601
Data columns (total 4 columns):
calls       2571 non-null float64
minutes     2571 non-null float64
messages    2571 non-null float64
mb_used     2571 non-null float64
dtypes: float64(4)
memory usage: 100.4 KB


In [9]:
features_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 2424 to 873
Data columns (total 4 columns):
calls       643 non-null float64
minutes     643 non-null float64
messages    643 non-null float64
mb_used     643 non-null float64
dtypes: float64(4)
memory usage: 25.1 KB


In [10]:
# < разделяю данные на валидационную и тестовую выборки >
df_train, df_test = train_test_split(df_train, test_size=0.20, stratify = df_train['is_ultra'], random_state=12345) 
# < создаю переменные для признаков и целевого признака >
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra'] 

In [11]:
features_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 2424 to 873
Data columns (total 4 columns):
calls       643 non-null float64
minutes     643 non-null float64
messages    643 non-null float64
mb_used     643 non-null float64
dtypes: float64(4)
memory usage: 25.1 KB


In [12]:
features_valid

,calls,minutes,messages,mb_used
2424,56.0,364.36,0.0,14129.94
2118,76.0,430.70,34.0,25138.49
2034,108.0,812.04,4.0,24490.28
2286,67.0,460.76,27.0,16626.26
1364,69.0,515.52,34.0,21231.90
...,...,...,...,...
2295,10.0,61.30,0.0,958.19
2327,58.0,370.03,29.0,33526.69
1983,44.0,293.54,51.0,11816.92
2315,48.0,341.65,0.0,19000.02


In [13]:
features_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 515 entries, 1593 to 1716
Data columns (total 4 columns):
calls       515 non-null float64
minutes     515 non-null float64
messages    515 non-null float64
mb_used     515 non-null float64
dtypes: float64(4)
memory usage: 20.1 KB


In [14]:
features_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2056 entries, 2406 to 1439
Data columns (total 4 columns):
calls       2056 non-null float64
minutes     2056 non-null float64
messages    2056 non-null float64
mb_used     2056 non-null float64
dtypes: float64(4)
memory usage: 80.3 KB


## 3. Исследование модели

### Проверяю 3 модели :
- Случайный лес
- Дерево решений
- Логистическая регрессия

### Модель "Случайный лес"

In [15]:
SL_best_model = None
SL_best_result = 0
SL_best_est = 0
SL_best_depth = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        SL_model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        SL_model.fit(features_train, target_train) 
        SL_predictions_valid = SL_model.predict(features_valid) 
        SL_result = accuracy_score(target_valid, SL_predictions_valid)  
        if SL_result > SL_best_result:
            SL_best_model = SL_model
            SL_best_result = SL_result
            SL_best_est = est
            SL_best_depth = depth

print("Accuracy наилучшей модели:", SL_best_result, "Количество деревьев:", SL_best_est, "Максимальная глубина:", depth)

Accuracy наилучшей модели: 0.8227060653188181 Количество деревьев: 50 Максимальная глубина: 10


### Модель "Дерево решений"

In [16]:
DR_best_model = None
DR_best_result = 0
for depth in range(1, 6):
    DR_model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    DR_model.fit(features_train, target_train) 
    DR_predictions_valid= DR_model.predict(features_valid) 
    DR_result = accuracy_score(target_valid, DR_predictions_valid) 
    if DR_result > DR_best_result:
        DR_best_model = DR_model
        DR_best_result = DR_result
        DR_best_depth = depth
        
print("Accuracy наилучшей модели на валидационной выборке:", DR_best_result)
print("Наилучшее значение гиперпараметра:", DR_best_depth)

Accuracy наилучшей модели на валидационной выборке: 0.8040435458786936
Наилучшее значение гиперпараметра: 5


### Модель "Логистическая регрессия"

In [17]:
LR_model = LogisticRegression(random_state=12345)  
LR_model.fit(features_train, target_train) 
LR_predictions_valid= LR_model.predict(features_valid) 
LR_result = accuracy_score(target_valid, LR_predictions_valid) 

print("Accuracy модели логистической регрессии на валидационной выборке:", LR_result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7122861586314152


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## 4. Проверка модели на тестовой выборке

In [18]:
SL_result_test = 0
SL_predictions_test = SL_model.predict(features_test) 
SL_result_test = accuracy_score(target_test, SL_predictions_test)  
 
print("Accuracy наилучшей модели на тестовой выборке:", SL_result_test)

Accuracy наилучшей модели на тестовой выборке: 0.8252427184466019


## 5. Проверка модели на адекватность

In [21]:
x = []
for i in range(len(target_test)):
    x.append(0)
    
target_pred_constant = pd.Series(x)
SL_result_dop = 0
SL_result_dop = accuracy_score(target_pred_constant, target_test) 
 
print("Accuracy:", SL_result_dop)

Accuracy: 0.6932038834951456


### Выводы:

По результатам исследования было выявлено, что лучшей моделью для предсказания значения значений является модель "Случайный лес". Для нее было проверено 6 разных значений гиперапараметра - глубина. Наилучшим значением оказалось значение 10.
Была проведена проверка этой модели на тестовой выборке. Точность составила почти 83%. Проверила модель на адекватность - модель является адекватной, так как ее точность выше, чем точность константной модели.